In [1]:
using CausalForest
using RCall
using StatsBase
using Statistics
using Random
using Distributions

In [2]:
function get_all_nodes_in_tree!(
    tree ,
    depth = 3,
    result  = []
    )
    if hasproperty(tree, :featid) && depth > 0
        push!(result, tree.featid)
        get_all_nodes_in_tree!(tree.left, depth-1, result)
        get_all_nodes_in_tree!(tree.right, depth-1, result)
    end
    return result
end

function get_freq(forest, depth=3) 
    ensemble = forest.trees
    n_trees = length(ensemble)
    res = []
    for i = 1:n_trees
        append!(res, get_all_nodes_in_tree!(forest.trees[i].tree, depth))
    end
    return proportionmap(res)
end

get_freq (generic function with 2 methods)

In [15]:
Random.seed!(123);
n, m = 10^4, 3;
@rlibrary grf
R"""
set.seed(123)
res <- data.frame()
"""
errors_grf = zeros(1)
errors_hterf = zeros(1)
hterf1 = zeros(1)
grf1 = zeros(1)
for j in 1:1
    u = Uniform(-pi,pi);
    features = rand(u, (n, m));
    X = features;
    b = Bernoulli();
    T = convert(Vector{Int64},rand(b, n));
    Y = cos.(features[:,2])+sin.(features[:,1]).*(T.+2).^3
    Xtest = rand(u, (n, m));
    tau = 19*sin.(Xtest[:,1])
    @rput X T Y Xtest tau
    R"""
    cf <- grf::causal_forest(X, Y, T, num.trees=500, tune.num.trees=500, sample.fraction=0.7, ci.group.size=1)
    tau.hat <- predict(cf, Xtest)$predictions
    mse = sqrt(mean((tau.hat - tau)^2))
    g_1 = grf::variable_importance(cf)[1]
    """
    @rget mse g_1 
    cf = build_forest(false, true, Y, T, X, true, m, 500, 500)
    tauhat = apply_forest(cf, Xtest)
    hterf1[j] = importance(cf)[1]
    errors_hterf[j] = rmsd(tau, tauhat)
    grf1[j] = g_1
    errors_grf[j] = mse
end
err_hterf = mean(errors_hterf)
err_grf = mean(errors_grf)
grf_1 = mean(grf1)
hterf_1 = mean(hterf1)
@rput err_grf err_hterf grf_1 hterf_1
R"""
dfgrf = data.frame(method = "GRF", RMSE = err_grf, imp1 = grf_1)
dfhterf = data.frame(method = "HTERF", RMSE = err_hterf, imp1 = hterf_1)
res = rbind(res, dfgrf, dfhterf)

"""
@rget res
print(res)

2×3 DataFrame
 Row │ method  RMSE      imp1     
     │ String  Float64   Float64  
─────┼────────────────────────────
   1 │ GRF     0.320771  0.77728
   2 │ HTERF   0.208716  0.999835